## Modeling

Python version spiral ... 
- > tf 2.11.0 introduced changes that break recommenders integration, need to step back in time 
- pyenv install 3.9
- pyenv virtualenv 3.9 recommenders
- pyenv uninstall recommenders
- pyenv activate recommenders

In [ ]:
%set_env TF_USE_LEGACY_KERAS=True

In [6]:
import pandas as pd

In [7]:
reviews = pd.read_parquet("../data/processed/reviews_small.parquet")
items = pd.read_parquet("../data/processed/items_small.parquet")

In [8]:
reviews

,rating,item_id,user_id,timestamp
2458,5,B07SMNZK8H,AGJY4LG74YJEJBVWIPIMIZ464ELA,2020-08-08 18:36:03.807
2459,5,B081B15L2D,AGJY4LG74YJEJBVWIPIMIZ464ELA,2020-01-15 02:13:23.360
2460,4,B076WXJPQ1,AGJY4LG74YJEJBVWIPIMIZ464ELA,2020-01-15 02:07:49.114
2461,5,B08KZ1TZYB,AGJY4LG74YJEJBVWIPIMIZ464ELA,2020-01-15 01:37:19.189
2462,5,B07961C64Q,AGJY4LG74YJEJBVWIPIMIZ464ELA,2020-01-15 01:34:23.669
...,...,...,...,...
9992866,2,B08LKZ27C9,AGZD4JETABL3LFXWWHAKX4LTZLYA,2021-09-17 02:12:14.502
9992867,2,B0BVH2SZRZ,AGZD4JETABL3LFXWWHAKX4LTZLYA,2021-08-08 23:14:55.627
9992868,3,B002GAXWFW,AGZD4JETABL3LFXWWHAKX4LTZLYA,2019-02-02 02:34:43.866
9992869,4,B013PKPP24,AGZD4JETABL3LFXWWHAKX4LTZLYA,2015-12-15 20:10:16.000


In [9]:
items

,title,average_rating,rating_number,description,price,images,item_id
2,Digi-Tatoo Decal Skin Compatible With MacBook ...,4.5,246,[],19.99,[{'hi_res': 'https://m.media-amazon.com/images...,B07SM135LS
3,NotoCity Compatible with Vivoactive 4 band 22m...,4.5,233,[],9.99,[{'hi_res': 'https://m.media-amazon.com/images...,B089CNGZCW
6,"QGHXO Band for Garmin Vivofit 4, Soft Silicone...",4.4,707,"[Compatibility, Custom designed for your preci...",14.89,[{'hi_res': 'https://m.media-amazon.com/images...,B07BJ7ZZL7
12,NANW Bands Compatible with Fitbit Versa/Versa ...,4.6,4249,[],None,[{'hi_res': 'https://m.media-amazon.com/images...,B07PXWYLQX
13,eForCity Leather Case with Stand for 7-Inch Sa...,3.9,144,[Compatible With Samsung Galaxy Tab 2 P3100 7....,None,"[{'hi_res': None, 'large': 'https://m.media-am...",B00ABEQ6TY
...,...,...,...,...,...,...,...
1609786,14-15 Inch Waterproof Laptop Tablet Sleeve Bag...,4.6,109,"[360° Protective Laptop Sleeve Case Bag, Ultra...",None,[{'hi_res': 'https://m.media-amazon.com/images...,B0817QF9BH
1609816,Lekaby Wireless Earbuds Bluetooth 5.3 Headphon...,5.0,162,[],None,[{'hi_res': 'https://m.media-amazon.com/images...,B0BZ8K2LJX
1609838,Synology DiskStation DS213+,4.3,137,[DiskStation DS213+],None,[{'hi_res': 'https://m.media-amazon.com/images...,B0091MHMZ4
1609860,"16GB Flash Drive, JEVDES 10 Pack Swivel Data S...",4.6,130,[],None,[{'hi_res': 'https://m.media-amazon.com/images...,B085L8BYPW


In [10]:
def extract_users(reviews): 
    """
    Given reviews, generate a user dataframe     
    """
    users = reviews.groupby(['user_id']).rating.count()
    users = pd.DataFrame(users).reset_index()
    users.rename(columns={'rating':'ratings'}, inplace=True)
    return users 

In [11]:
users = extract_users(reviews)

In [12]:
all_items = set(items.item_id)
reviews = reviews[reviews.item_id.isin(all_items)]

❗in the notebook, ratings are thresholded ... do we need to follow suit? what are the ramifications if we don't? OH... in the notebook, a click is an interaction, there's no middle ground. the network is going to operate on 0s or 1s. by leaving low reviews in our matrix, the network would learn to recommend things users have interacted with, but not necessarily positively. our case is the same, a review is an interaction. we're aiming to recommend, and we should not want to recommend low reviews. so filter... 

In [13]:
# Discard reviews by users outside our core group 
reviews = reviews[reviews.user_id.isin(set(users.user_id))]

In [14]:
reviews

,rating,item_id,user_id,timestamp
2458,5,B07SMNZK8H,AGJY4LG74YJEJBVWIPIMIZ464ELA,2020-08-08 18:36:03.807
2459,5,B081B15L2D,AGJY4LG74YJEJBVWIPIMIZ464ELA,2020-01-15 02:13:23.360
2460,4,B076WXJPQ1,AGJY4LG74YJEJBVWIPIMIZ464ELA,2020-01-15 02:07:49.114
2461,5,B08KZ1TZYB,AGJY4LG74YJEJBVWIPIMIZ464ELA,2020-01-15 01:37:19.189
2462,5,B07961C64Q,AGJY4LG74YJEJBVWIPIMIZ464ELA,2020-01-15 01:34:23.669
...,...,...,...,...
9992866,2,B08LKZ27C9,AGZD4JETABL3LFXWWHAKX4LTZLYA,2021-09-17 02:12:14.502
9992867,2,B0BVH2SZRZ,AGZD4JETABL3LFXWWHAKX4LTZLYA,2021-08-08 23:14:55.627
9992868,3,B002GAXWFW,AGZD4JETABL3LFXWWHAKX4LTZLYA,2019-02-02 02:34:43.866
9992869,4,B013PKPP24,AGZD4JETABL3LFXWWHAKX4LTZLYA,2015-12-15 20:10:16.000


In [15]:
import numpy as np 
import pandas as pd 

### Naive

In [16]:
# TODO

### Nearest Neighbor

In [18]:
from recommenders.datasets.sparse import AffinityMatrix

In [20]:
reviews = reviews.drop(['timestamp'], axis=1)

In [21]:
header = {
    "col_user": "user_id",
    "col_item": "item_id",
    "col_rating": "rating",
}
ui_sparse = AffinityMatrix(reviews, **header)

# This isn't implied by the name, but this densifies the matrix, i.e. we have a contiguous u x i
# matrix here (user vector of item ratings) ... though it's actually not clear how the memory is 
# managed underneath in scipy, the 'dense' array might just be a bunch of pointers to the DFs stored 
# in the AM object... 
ui_dense, u_map, i_map = ui_sparse.gen_affinity_matrix()

In [22]:
import sys 
print(sys.getsizeof(ui_sparse))
print(sys.getsizeof(ui_sparse.df))
print(sys.getsizeof(ui_dense))

48
28332216
128


In [23]:
u_map

{'AE22ICKM5TVDWFZTKENDXYXAG4HQ': 0,
 'AE22SINAPMHADBSGN4FGFE3AA2KQ': 1,
 'AE23U5BF4DTI7H5ZVX6L5M3U7FUQ': 2,
 'AE23ZKCFLF7QXF7D4DAXZ6OTRKJA': 3,
 'AE257VA6SYLLY444MOAZ2YKC3SGA': 4,
 'AE25GSBHXO3S6MSOLKPWP33O54WQ': 5,
 'AE25MINY74XBAC2LYMUXMR4P3F4A': 6,
 'AE25XOZI6QP5324ICXPSEMWSI65Q': 7,
 'AE2662PXKHDBYWPJF54F4BM6LW4Q': 8,
 'AE26ACRB3FRFTAK44HF6WMYHCHRA': 9,
 'AE26VCF2CYWWOJEDAEZWKAFRK5MQ': 10,
 'AE26X3GVKQECDJDCV2H3CI5WMKNA': 11,
 'AE272WWGUOKE3ZOUNDPQOZT3OXBA': 12,
 'AE27S24FRRZM7V366TUY6TOIM6PA': 13,
 'AE2A3WTEPJ2GYCV4XN3TQOT7642A': 14,
 'AE2ABWG2JAVB6GVTNKSB34ULQVUA': 15,
 'AE2B5XQMDFZ7EKKEKTJQYFQ6JXPA': 16,
 'AE2BNCSTTZE2A72FOKHEHVYGGNLA': 17,
 'AE2CO6MCKFITWP2QMMS2HZAVXD7A': 18,
 'AE2CUBSQ2HBH6I5F7QCKPRWUR44Q': 19,
 'AE2D6MLZFOE6I36JDWPHC2ZA5UJQ': 20,
 'AE2DT43OAV25ZLHZ6TVGHVDT6NKQ': 21,
 'AE2DTAQSG2KFUREVLBMUNPQEQDMA': 22,
 'AE2DYM7CZLXHZ2JCGSOYL7ZKZPIA': 23,
 'AE2EIVRSPIHATIYULGHMFG456XKQ': 24,
 'AE2EXGCAZGRJNHIXMPK5QMATRCHA': 25,
 'AE2F7LZHYFSN7U27QW7NBYIBCBXQ': 26,
 'AE2FQ754O

In [24]:
len(u_map)

10000

In [25]:
ui_dense[0] 

array([5, 5, 5, ..., 0, 0, 0])

In [26]:
ui_dense[1][[0,413]]

array([0, 0])

In [ ]:
list_ = np.nonzero(ui_dense[0])[0]
b = np.nonzero(ui_dense[1])[0]

In [ ]:
np.concatenate([list_, b], axis=0) 

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22])

In [29]:
ui_dense[0][np.nonzero(ui_dense[0])]

array([5, 5, 5, 5, 5, 5, 5, 4, 2, 5, 5, 5])

In [ ]:
list_ = set(np.nonzero(ui_dense[0])[0]) 
b = set(np.nonzero(ui_dense[6])[0]) 

In [ ]:
list_

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [ ]:
np.max(list_)

np.int64(11)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
sim_a = cosine_similarity([ui_dense[0]], [ui_dense[33]])
sims = [0] *len(users)

In [ ]:
sims[0] = sim_a

In [ ]:
top_k = 5

In [ ]:
[[0]*2]*top_k 

In [168]:
fake_ratings = np.array([ np.random.random() for x in range(100) ]) 

In [169]:
def find_key(dict_, val): 
    for k, v in dict_.items(): 
        if v == val: 
            return k

In [170]:
def argmax_(list_, exclude): 
    """Argmax with a list of indices to include, and exclude"""    
    max_i = 0 
    for i, a in enumerate(list_): 
        if i not in exclude: 
            if a > list_[max_i]: 
                max_i = i 
    return max_i 

In [171]:
preds = []
ui = ui_dense 

for u in range(len(u_map)): 
    rated = list(np.nonzero(ui[u])[0]) 
    recommended = []
    while len(recommended) < 10: 
        best_rated = argmax_(fake_ratings, exclude=rated + recommended)
        recommended.append(best_rated) 

    recommendations = [ { find_key(i_map, x): fake_ratings[x] } for x in recommended ]
    preds.append(recommendations)


In [176]:
preds[5]

[{'B01LXFFO8O': np.float64(0.9964913244381501)},
 {'B074CV39CZ': np.float64(0.9832080627781924)},
 {'B004VM1T5S': np.float64(0.9675099580232271)},
 {'B00UVSJU4S': np.float64(0.9586008552468466)},
 {'B0819W2H7V': np.float64(0.9547344230493247)},
 {'B0082E9K7U': np.float64(0.9339118106753722)},
 {'B00NUB333W': np.float64(0.930857640711958)},
 {'B0BYRMDDCP': np.float64(0.8828239187931431)},
 {'B00QDF34UO': np.float64(0.8783519687827609)},
 {'B019G1QAXY': np.float64(0.8657701458766286)}]

In [177]:
rated

[np.int64(758),
 np.int64(1062),
 np.int64(2834),
 np.int64(18321),
 np.int64(50205),
 np.int64(57059)]

In [149]:
fake_ratings

array([0.79972592, 0.96773651, 0.03627581, ..., 0.56387805, 0.10178226,
       0.52135765])

In [132]:
m = np.argmax(fake_ratings[candidates])
fake_ratings[m]

np.float64(0.9380747051543531)

In [122]:

f = fake_ratings[:10]
f

array([0.79972592, 0.96773651, 0.03627581, 0.94500918, 0.6972711 ,
       0.10736279, 0.46726819, 0.34114489, 0.29949057, 0.92829034])

In [126]:
np.argmax(fake_ratings)

np.int64(7830)

In [ ]:
recommended

[{'B017M5T894': np.float64(0.44170344314473553)},
 {'B098S85M4X': np.float64(0.37548236440174165)},
 {'B097MSG8LM': np.float64(0.47973793534366904)},
 {'B01KUEGE1E': np.float64(0.7132452507268774)},
 {'B0B1M66ZGQ': np.float64(0.34813045609918136)},
 {'B00B9Z23NM': np.float64(0.9766051685195317)},
 {'B00ESYW40S': np.float64(0.8228181869014046)},
 {'B009D7BJR4': np.float64(0.20749694824672038)},
 {'B00B97KDV4': np.float64(0.9558561656733956)},
 {'B0098OCKDE': np.float64(0.7962374139470548)}]

In [91]:
fake_ratings[[13, 5]]

array([0.22362651, 0.10736279])

In [85]:
ui[-2].argmax()

np.int64(1052)

In [ ]:
from scipy.stats import pearsonr 

def pearson_similarity(a, b):
    """
    Compute Pearson similarity
    """
    return (1 + pearsonr(a, b).statistic) / 2

In [ ]:
# Populating a full user similarity matrix is inherently limited by (and is a questionable 
# strategy because of) the u^2 memory requirement. Unlike our affinity matrices, 
# these are not sparse. Since our goal is recommending items, we'll compute the similarity
# iterativealy and store the most similar users to get down to C * u memory
similarity_matrix = np.array([[0.] * top_k] * len(users))
for list_ in range(len(users)): 

    # Collect our similarities w/ respect to user A
    sim_a = {}
    for b in range(len(users)): 
        if list_ != b: 
            
            # Given the sparsity of our review vectors, cosine similarity is going to be 
            # effectively zero if we look across the entire item space... compare only those 
            # items these two users have in common (at least 1 rating between the two). 
            a_item_ix = np.nonzero(ui_dense[list_])[0]            
            b_item_ix = np.nonzero(ui_dense[b])[0]
            all_ix = np.concatenate([a_item_ix, b_item_ix])
            a_items = ui_dense[list_][all_ix]            
            b_items = ui_dense[b][all_ix]

            # Fill non-ratings with middling scores. Non-interactions appear 
            # dissimilar to positive reviews and similar to negative ones otherwise.
            a_items[a_items==0] = 4
            b_items[b_items==0] = 2
            
            # Cosine similarity risks insensitivity to rating value, while imperfect here, 
            # Pearson similarity gets us sensitivty to rating magnitude and trends
            sim_a[b] = pearson_similarity(a_items, b_items)
        
    # Find and store the top k user matches, in order    
    # NOTE: dict sorting logic courtesy of gpt-4o (https://chatgpt.com/share/687dc72f-54b4-8013-806e-b1de20d0ef12)
    top = sorted(sim_a.items(), key=lambda x: x[1], reverse=True)[:top_k]
    similarity_matrix[list_] = [x[0] for x in top]

    break 

In [ ]:
#a_items[-7] = 2
a_items

In [ ]:
b_items

In [ ]:
cosine_similarity([a_items],[b_items])

In [ ]:
pearson_similarity(a_items, b_items)

### VAE

In [ ]:
from recommenders.datasets.sparse import AffinityMatrix

In [ ]:
# Only relevant to VAE strategy 
reviews = reviews[reviews.rating >= 3]

In [ ]:
reviews.columns

In [ ]:
reviews

In [ ]:
# NOTE: Strategy adapted from tutorials available in the Recommenders project, see 
# https://github.com/recommenders-team/recommenders/tree/main
from recommenders.datasets.python_splitters import python_random_split

# Split along user boundaries to ensure no leakage of preference between train and test
train_users, test_users, val_users = python_random_split(users, [.9, .05, .05])

In [ ]:
print(train_users.shape, test_users.shape, val_users.shape)

In [ ]:
train = reviews[reviews.user_id.isin(train_users.user_id)]
val = reviews[reviews.user_id.isin(val_users.user_id)]
test = reviews[reviews.user_id.isin(test_users.user_id)]

In [ ]:
print(train.shape, val.shape, test.shape)

In [ ]:
# Technique from Recommenders (see https://github.com/recommenders-team/recommenders/blob/45e1b215a35e69b92390e16eb818d4528d0a33a2/examples/02_model_collaborative_filtering/standard_vae_deep_dive.ipynb) 
# to improve utility of validation set during training - only allow items in
# the validation set that are also present in the train set
val = val[val.item_id.isin(train.item_id.unique())]

In [ ]:
val.shape

In [ ]:
from recommenders.datasets.python_splitters import python_stratified_split 

In [ ]:
# Another technique employed in Recommenders (see above link for notebook), for in-flight validation to be 
# meaningful during training, our validation set needs not just ground truth, but unseen validation samples 
# to see if predictions for validation users are relevant (to those users). Anyway, break down our val and test 
# sets again to support this strategy
val_src, val_target = python_stratified_split(
    data=val, 
    ratio=0.8, 
    filter_by="item", 
    col_user="user_id", 
    col_item="item_id"
    )
test_src, test_target = python_stratified_split(
    data=test, 
    ratio=0.8, 
    filter_by="item", 
    col_user="user_id", 
    col_item="item_id"
    )

In [ ]:
print(val.shape, " -> ", val_src.shape, val_target.shape)
print(test.shape, " -> ", test_src.shape, test_target.shape)

In [ ]:
#to use standard names across the analysis 
header = {
        "col_user": "user_id",
        "col_item": "item_id",
        "col_rating": "rating",
        # Unclear why this doesn't also eat a timestamp, but many of the functions that split temporally use, fortunately 
        # the column 'timestamp' (i.e. DEFAULT_TIMESTAMP_COL='timestamp') so I think we're fine. 
        # "col_timestamp" : "timestamp"
    }

train_matrix = AffinityMatrix(df=train, **header)
val_matrix = AffinityMatrix(df=val, **header)
val_src_matrix = AffinityMatrix(df=val_src, **header)
val_tgt_matrix = AffinityMatrix(df=val_target, **header)
test_src_matrix = AffinityMatrix(df=test_src, **header)
test_tgt_matrix = AffinityMatrix(df=test_target, **header)

In [ ]:
# This generates a sparse array of user vectors, aka user-item matrix
# X[0] is the first user in the list, with entries for all items known when the matrix was constructed in that row
train, _, _ = train_matrix.gen_affinity_matrix()
val, _, _ = val_matrix.gen_affinity_matrix() 
val_src, _, _ = val_src_matrix.gen_affinity_matrix()
val_tgt, _, _ = val_tgt_matrix.gen_affinity_matrix()
test_src, _, _ = test_src_matrix.gen_affinity_matrix()
test_tgt, _, _ = test_src_matrix.gen_affinity_matrix()

In [ ]:
from recommenders.utils.python_utils import binarize

train = binarize(train, 3)
val = binarize(train, 3)
val_src = binarize(val_src, 3) 
val_tgt = binarize(val_tgt, 3)
test_src = binarize(test_src, 3)
test_tgt = binarize(test_tgt, 3)

In [ ]:
# TODO: Make sure this is reported during training/configuration
sparsity = np.count_nonzero(train)/(train.shape[0]*train.shape[1])*100
print(f"sparsity: {sparsity:.2f}%")

In [ ]:
keras.__version__ 

In [22]:
from recommenders.models.vae.standard_vae import StandardVAE

2025-07-21 20:43:56.923222: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-21 20:43:57.118547: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753145037.183633 1891196 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753145037.208377 1891196 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753145037.373871 1891196 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

ModuleNotFoundError: No module named 'tensorflow.keras'

In [ ]:
model = StandardVAE(
    n_users = train.shape[0], 
    original_dim = train.shape[1],
    intermediate_dim=250, 
    latent_dim=50, 
    n_epochs=1, 
    batch_size=1, 
    k=10, 
    verbose=1, 
    seed=4, 
    save_path="models/svae.hdf5", 
    drop_encoder=0.5, 
    drop_decoder=0.5, 
    annealing=False, 
    beta=1.0) 

In [ ]:
type(train)

In [ ]:
model.fit(
    x_train=train, 
    x_valid=val, 
    x_val_tr=val_src, 
    x_val_te=val_tgt, 
    mapper=val_matrix,
    )

Managing text-based reviews at this scale could be a challenge, and I'd like to steer clear of LLMs for this effort. We could do an embedding on the review and use that for similarity, but we have pretty rich item data. Perhaps let's ignore the collaborative aspect here and build a shopping interface that: 
- surfaces the most popular items, and encourages you to add items to your shopping cart for a big discount/promo
- based on clicks and cart items, improves the recommendations and surfaces new products

We can use an autoencoder to accept a sparse matrix of users and items, learn to reproduce that matrix, and in so doing support prediction on missing values. However, this matrix is of size users x items, which here is 1.8e7 x 1.6e6 = 28,125,000,000 KB (best-case, higher if stored as np floats) ~= 26 TB !! WTF. 
- In the standard VAE example (https://github.com/recommenders-team/recommenders/blob/main/examples/02_model_collaborative_filtering/standard_vae_deep_dive.ipynb) the clicks are turned into a histogram for each user ... so we have n_user vector of length n_items... then I guess each of these is a training sample. The VAE presumably learns, given a sparse user vector, to predict every rating. This takes the complexity down and gives us a training set we can iterate over. 

Let's avoid any distributional pressure (present in VAE, SVAE, disentangled VAE) and go for a basic autoencoder using the strategy laid out above, i.e.

### Scratch Autoencoder

That damned Recommenders VAE is a dependency dumpster fire ... walking away after 10h of fighting crusty environments that generate more errors than outcomes. Shift to a basic autoencoder in pytorch and just eat the cost of having to implement our own validation. 

In [ ]:
users

In [ ]:
reviews

In [ ]:
items

In [24]:
import os
import math 
import torch 
import pandas as pd 
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import wandb
import sys
sys.path.append('..')
from recommenders.datasets.sparse import AffinityMatrix
from recommenders.datasets.python_splitters import python_random_split
from recommenders.datasets.python_splitters import python_stratified_split 

In [ ]:
class Autoencoder(nn.Module):
    """
    Autoencoder

    NOTE: with cues from https://www.geeksforgeeks.org/deep-learning/implementing-an-autoencoder-in-pytorch/
    """

    def __init__(self, dims=1000):
        """
        Initialize a new object given an item count 
        """
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Linear(dims, 500),
            nn.ReLU(), 
            nn.Linear(500, 75),
            nn.ReLU(), 
        )
        self.decoder = nn.Sequential(
            nn.Linear(75, 500),
            nn.ReLU(), 
            nn.Linear(500, dims),
            nn.ReLU(), 
        )

    def forward(self, x):
        """
        Implement our forward pass 
        """
        h = self.encoder(x) 
        r = self.decoder(h)

        return r

In [ ]:
class DeepCartDataset(torch.utils.data.Dataset): 
    """
    Custom pytorch-compatible dataset. Adapted from 
    https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files
    """
    def __init__(self, users, reviews): 
        """
        Initialize a new instance

        Oof. The ideal pattern here is for the dataset to be blissfully ignorant of our split strategy and just 
        make a dataset available to its client based on the raw data passed. However the split strategy is rather intricate below ... 
        note the five splits. Can we easily raise that up to a higher level? The refactoring might not be trivial and it may result in 
        residue of this split strategy bleeding over to the other models -- what's common and what's not? 

        common: 
        - train, val, test split
        - val test and test test bonus splits - for all validation stages we need to check performance, however non-NN techniques 
          really only need a test split, right? if we do train holdout for validation, we essentially have three test sets (train-test, val, test)
        - a need to operate on the same validation or at least test data, lest the comparison be biased by the selection method each model applies

        unique
        - logic to prune reviews < 3.5 -- we don't do this in cfnn, and naive doesn't care (predicts highest review in the matrix), if this is 
          done during training, it will also need to be done during inference
        - need for a pytorch-style dataset ... the naive method is doing a O(n) search, the cfnn needs dataframes -- while refactoring is 
          possible, why understake the risk it will be a disjoint and inelegant fit? 
        - the VAE implementation wants all train and val, but doesn't require a test dataset. we ou


        we could: 
        - pass train and val, hold test out
        - pass test to predict function, which we need for the demo anyway
        - keep the pytorch dataset unique to the pytorch-compatible class... doesn't make sense to try and foist on other algos... we are 
        doing this in the wrong order, filtering and then splitting... we need to outsource the splitting and then do the filtering inside each 
        model 

        right now this is speculation, just get something working! we can figure out how to streamline after -- oh, but we need a dataset 
        implementation
        """
        self.users = users 
        self.reviews = reviews 
        self.matrix = 

    def build_affinity_matrices(): 
        """
        
        """
        oof

    def split(users, reviews, items):
        """
        Generate splits 
        """
        print(f"Full user-item matrix is {len(users) * len(items)}")



# generic need
        # NOTE: Strategy adapted from tutorials available in the Recommenders project, see 
        # https://github.com/recommenders-team/recommenders/tree/main
        # Split along user boundaries to ensure no leakage of preference between train and test
        train_users, test_users, val_users = python_random_split(users, [.9, .05, .05])
        print(train_users.shape, test_users.shape, val_users.shape)

        train = reviews[reviews.user_id.isin(train_users.user_id)]
        val = reviews[reviews.user_id.isin(val_users.user_id)]
        test = reviews[reviews.user_id.isin(test_users.user_id)]
        print(train.shape, val.shape, test.shape)



        # Technique from Recommenders (see https://github.com/recommenders-team/recommenders/blob/45e1b215a35e69b92390e16eb818d4528d0a33a2/examples/02_model_collaborative_filtering/standard_vae_deep_dive.ipynb) 
        # to improve utility of validation set during training - only allow items in
        # the validation set that are also present in the train set
        val = val[val.item_id.isin(train.item_id.unique())]
        print(val.shape)

        # Another technique employed in Recommenders (see above link for notebook), for in-flight validation to be 
        # meaningful during training, our validation set needs not just ground truth, but unseen validation samples 
        # to see if predictions for validation users are relevant (to those users). Anyway, break down our val and test 
        # sets again to support this strategy
        val_src, val_target = python_stratified_split(
            data=val, 
            ratio=0.8, 
            filter_by="item", 
            col_user="user_id", 
            col_item="item_id"
            )
        test_src, test_target = python_stratified_split(
            data=test, 
            ratio=0.8, 
            filter_by="item", 
            col_user="user_id", 
            col_item="item_id"
            )
        
        print(val.shape, " -> ", val_src.shape, val_target.shape)
        print(test.shape, " -> ", test_src.shape, test_target.shape)

# really all of these models just need the review data to train, the users and item lists can be extracted from that

# AE specific ... 
        # We are trying to teach the model what a good interaction is like, and we'll 
        # ultimately be interested only in whether to recommend an item or not ... 
        # low reviews are not something we want the model suggesting... 
        reviews_low = reviews[reviews.rating < 3]
        reviews = reviews[reviews.rating >= 3]

        header = {
            "col_user": "user_id",
            "col_item": "item_id",
            "col_rating": "rating",
        }

        train_matrix = AffinityMatrix(df=train, **header)
        val_matrix = AffinityMatrix(df=val, **header)
        val_src_matrix = AffinityMatrix(df=val_src, **header)
        val_tgt_matrix = AffinityMatrix(df=val_target, **header)
        test_src_matrix = AffinityMatrix(df=test_src, **header)
        test_tgt_matrix = AffinityMatrix(df=test_target, **header)

        # This generates a sparse array of user vectors, aka user-item matrix
        # X[0] is the first user in the list, with entries for all items known when the matrix was constructed in that row
        train, _, _ = train_matrix.gen_affinity_matrix()
        val, _, _ = val_matrix.gen_affinity_matrix() 
        val_src, _, _ = val_src_matrix.gen_affinity_matrix()
        val_tgt, _, _ = val_tgt_matrix.gen_affinity_matrix()
        test_src, _, _ = test_src_matrix.gen_affinity_matrix()
        test_tgt, _, _ = test_src_matrix.gen_affinity_matrix()    

        train = binarize(train, 3)
        val = binarize(train, 3)
        val_src = binarize(val_src, 3) 
        val_tgt = binarize(val_tgt, 3)
        test_src = binarize(test_src, 3)
        test_tgt = binarize(test_tgt, 3)

        sparsity = np.count_nonzero(train)/(train.shape[0]*train.shape[1])*100
        print(f"sparsity: {sparsity:.2f}%")
    def __len__(self): 
        """
        Retrieve length of the dataset
        """
        return len(self.img_labels) 
    
    def __getitem__(self, idx): 
        """
        Retrieve an item at the provided index
        """
        #TODO: implement
        pass

In [ ]:
def get_data_loader(batch_size=5, shuffle=True): 
    """
    Retrieve a pytorch-style dataloader 
    """

    #TODO: implement
    #transform = transforms.Compose([
    #     transforms.ConvertImageDtype(torch.float),
    #     transforms.Normalize(mean=[0.5], std=[0.5])
    #])

    #data = DeepCartDataset(transform=transform)
    #loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=shuffle)
    
    #return loader
    pass

In [ ]:
def train(loader, model, loss_interval=20, epochs=2, lr=0.01, momentum=0.9):
    """
    Train the model with the provided dataset

    NOTE: this is a similar training loop as we used for our vision model in the 
    the vision project, forward pass
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    train_loss = []

    tqdm.write(f"Starting training run...")    
    # TODO: configure WandB
    # see https://docs.wandb.ai/guides/integrations/pytorch/
    config = {}
    run = wandb.init(config=config) 

    model.train()
    model = model.to(device)
    
    loss_fn = nn.CrossEntropyLoss()

    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    for epoch in range(epochs):

        running_loss = 0.0
        for i, data in enumerate(loader):

            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)

            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            # collect metrics
            running_loss += loss.item()

            if (i % loss_interval) == (loss_interval - 1): 
                train_loss.append(running_loss / loss_interval)
                tqdm.write(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / loss_interval:.3f}")
                running_loss = 0 
    
    tqdm.write("Training complete!") 

    return train_loss 

In [ ]:
list_ = [[]] * 2

In [ ]:
list_[0].append('butt')

In [ ]:
list_

[['butt'], ['butt']]